# Initial Setup

- import various helpers, load data, select reviews by status and category

In [1]:
import sys
sys.path.append('../')

from application.name_obj_classes import PubName, PersonName, remove_punct

from application.review_obj_class import ReviewObj

from application.text_preprocessing import preprocess_text

import os
import pandas as pd
import re
from collections import Counter
import numpy as np
import pickle
from nltk.metrics import edit_distance
%pprint

Pretty printing has been turned OFF


In [2]:
from database import *
import database.models as models

# load full text from db
aps_details_single = models.Review().query.filter(models.Review.status.in_(('needs_crosscheck', 'done'))).filter(models.Review.review_type == 'single_focus').all()

len(aps_details_single)

561

In [27]:
titles = [i.reviewed_book_title for i in aps_details_single if i.reviewed_book_title != '' and i.reviewed_book_title is not None]
len(titles)

558

In [28]:
reviews_parsed = [ReviewObj(i.record_id, i.full_text) for i in aps_details_single if i.reviewed_book_title != '' and i.reviewed_book_title is not None]
#reviews_parsed[0].cleaned_text
#reviews_parsed[0].cleaned_toks[:10]
len(reviews_parsed)

558

In [6]:
import string
from nltk.util import ngrams
from collections import Counter
from nltk.corpus import stopwords

def remove_function_tail(sequence):
    if sequence[-1].lower() in stopwords.words('english'):
        sequence.pop()
        return remove_function_tail(sequence)
    else:
        return sequence

In [42]:
from fuzzysearch import find_near_matches

def match_title_candidates(text, known_titles):
    all_matches = {}
    for t in known_titles:
        #base fuzziness on length of pubname
        if len(t) < 10:
            fuzz=0
        elif len(t) > 9 and len(t) < 20:
            fuzz=1
        else:
            fuzz=3
        matches = find_near_matches(t, text, max_l_dist=fuzz)
        if len(matches) > 0:
            match_strings = [text[m.start:m.end] for m in matches]
            for i in match_strings:
                try:
                    all_matches[t].append(i)
                except:
                    all_matches[t] = [i,]
    for k,v in all_matches.items():
        if len(v) == 1 and k == v[0]:  
            return ("found", [k,])
        
    result = list(all_matches.keys())
    if len(result) > 0:
        return ("found", result)
    else:
        return ("not found", result)


In [43]:
match_title_candidates(reviews_parsed[3].no_pubs_text, titles)

('not found', [])

In [44]:
from collections import Counter 

all_title_matches = {}

# run on all reviews, return matches, novel, or no match
for review in reviews_parsed:
    
    result, title_candidates = match_title_candidates(review.cleaned_text, titles)
    
    if len(title_candidates) > 0:
        top_title_candidate = title_candidates[0]
    else:
        result = "not found"
        top_title_candidate = ''
    try:
        all_title_matches["type"].append(result) 
    except:
        all_title_matches["type"] = [result,]
    try:
        all_title_matches["results"].append(top_title_candidate)
    except:
        all_title_matches["results"] = [top_title_candidate,]
    try:
        all_title_matches["match_number"].append(len(title_candidates))
    except:
        all_title_matches["match_number"] = [len(title_candidates),]

In [51]:
import pandas as pd
df = pd.DataFrame(all_title_matches)
df['recorded_title'] = titles
len(df.loc[df['match_number'] == 0])/len(df)
#188 not found

0.33691756272401435

In [50]:
len(df.loc[df['results'] == df['recorded_title']])/len(df.loc[df['match_number'] == 1])

0.8498583569405099

In [ ]:
# 0.5376344086021505 of total identified
# 0.8498583569405099 of matches are correct and 0.33691756272401435 of total are unmatched
